# Ejercicio biosanitario: Diabetes (Pima) — Carga, limpieza y exportación

*Actualizado: 05/10/2025*

Pasos:
1) Cargar dataset desde URL.
2) Limpiar (ceros→NaN en variables clínicas, imputación mediana, duplicados, nombres).
3) Crear `bmi_cat` y `glucose_cat`.
4) Guardar en CSV y Excel e imprimir resumen.

> Aviso: categorías de glucosa solo educativas; no usar clínicamente.


## 0) Preparación

In [ ]:
!pip install -q pandas numpy openpyxl

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
URL = 'https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv'


## 1) Carga del dataset

In [ ]:
df = pd.read_csv(URL)
print('== CARGA INICIAL ==')
#nos devuelve el total de filas y columnas de nuestro dataset
print('Shape:', df.shape)
df.head() #indica cuantas filas quiero mostrar. Si lo dejamos vacío por defecto son 5


## 2) Limpieza

In [ ]:
#Esto limpia los nombres de las columnas del DataFrame df:
# str.strip() → elimina espacios al inicio y final.
# str.lower() → convierte todo a minúsculas.
#str.replace(' ', '_') → reemplaza los espacios por guiones bajos _

df.columns = (df.columns.str.strip().str.lower().str.replace(' ', '_', regex=False))

#Le indico de qué columnas quiero que se comprueben los datos
to_fix = ['glucose','bloodpressure','skinthickness','insulin','bmi']

#En esas columnas, reemplaza los ceros por valores faltantes (NaN).
#Esto es común en datos médicos donde un 0 no tiene sentido (ej. presión sanguínea no puede ser 0).
for c in to_fix:
    df[c] = df[c].replace(0, np.nan)

#Muestra cuántos valores NaN tiene cada una de esas columnas antes de “rellenarlos”.
print('Nulos antes de imputar:\n', df[to_fix].isna().sum(), '\n')

#Rellena los NaN (antes ceros) con la mediana de cada columna.
for c in to_fix:
    df[c] = df[c].fillna(df[c].median())

#Quita las filas duplicadas y muestra cuántas fueron eliminadas.
#Aquí se guarda en before cuántas filas tenía el DataFrame antes de eliminar duplicados.
before = df.shape[0]
#El método .drop_duplicates() elimina filas duplicadas completas del DataFrame.
#Por defecto, considera duplicadas aquellas filas donde todos los valores son idénticos en cada columna.
#Devuelve un nuevo DataFrame sin duplicados.
df = df.drop_duplicates()
removed = before - df.shape[0]
print('Duplicados eliminados:', removed)
df.head()


## 3) Columnas derivadas

In [ ]:
#A partir de la tabla original limpia de datos, estamos creando 3 columnas nuevas
def bmi_category(x):
    if x < 18.5: return 'bajo_peso'
    if x < 25: return 'normal'
    if x < 30: return 'sobrepeso'
    return 'obesidad'

def glucose_category(g):
    if g < 100: return 'normal'
    if g < 126: return 'prediabetes'
    return 'diabetes'

df['bmi_cat'] = df['bmi'].apply(bmi_category)
df['glucose_cat'] = df['glucose'].apply(glucose_category)
df[['bmi','bmi_cat','glucose','glucose_cat']].head()

print (df.head())

## 4) Guardar CSV y Excel

In [ ]:
Path('data').mkdir(exist_ok=True)
df.to_csv('data/pima_diabetes_limpio.csv', index=False, encoding='utf-8-sig')
df.to_excel('data/pima_diabetes_limpio.xlsx', index=False)
print('Guardado en data/pima_diabetes_limpio.csv y .xlsx')


## 5) Resumen

In [ ]:
print('Shape final:', df.shape)
print('\nNulos tras imputar:')
print(df[['glucose','bloodpressure','skinthickness','insulin','bmi']].isna().sum())
print('\nConteo bmi_cat:')
print(df['bmi_cat'].value_counts())
print('\nConteo glucose_cat:')
print(df['glucose_cat'].value_counts())
df.describe(include='all')


## 6) Gráfico Barras Conteo y Porcentaje


In [ ]:


import matplotlib.pyplot as plt

# Conteos y porcentajes
counts = df['bmi_cat'].value_counts().sort_index()
total = counts.sum()

# Crear figura
plt.figure(figsize=(8,5))  # 🔹 tamaño más amplio
ax = counts.plot(kind='bar', color='skyblue', edgecolor='black')  # 🔹 color suave y bordes

# Título y etiquetas
plt.title('Frecuencia por categoría de BMI', fontsize=14, fontweight='bold', pad=15)
plt.xlabel('Categoría BMI', fontsize=12)
plt.ylabel('Conteo', fontsize=12)

# 🔹 Rotación y alineación del texto en el eje X
plt.xticks(rotation=45, ha='right')

# Etiquetas encima de cada barra
for p in ax.patches:
    height = p.get_height()
    pct = 100 * height / total if total else 0
    ax.annotate(f'{int(height)}\n{pct:.1f}%',
                (p.get_x() + p.get_width()/2, height),
                ha='center', va='bottom',
                fontsize=10, color='black',
                xytext=(0, 5), textcoords='offset points')  # 🔹 separa el texto de la barra

# 🔹 Ajustar márgenes y cuadrícula
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

plt.show()
